In [196]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from functools import partial
import pandas as pd
from IPython.display import display
import ipywidgets as widgets

# 1. Simulation of Fernando's original model

In [231]:
#Params includes alpha, beta, theta, phi, lam, m in the final model

def model1(t, vars, params):
    alpha, beta, theta, phi, lam, m = params
    C_A2, C_S = vars
    dC_A2dt = 1 - alpha * C_A2 * C_S**m - theta * C_A2
    dC_Sdt = alpha/lam * C_A2 * C_S**m - beta * C_S**(m + 1) + theta/lam * C_A2 - phi * C_S
    return (dC_A2dt, dC_Sdt)

def model1_Hill(t, vars, params):
    pass

def calculate_all_conc(sol, consts):
    lam, m = consts
    C_A2 = sol.y[0]
    C_S = sol.y[1]
    C_M = C_S ** m
    C_A = 2 * (1 - C_A2) - lam * (C_S + C_M)
    return (C_A2, C_S + C_M, C_A)

## Control of oscillations (OOP version)

In [ ]:
#calc_others(sol, consts)
class oscmodel:
    def __init__(self, model, params, consts):
        self.model = model
        self.params = params
        self.consts = consts

        self.__calc_all = None
        self.species = ['cA2', 'cS_tol', 'cA']
        self.info = None
        self.exp_data = None
   
    def calc_all(self, function):
        self.__calc_all = function
        
    def simulate(self, init_cond, t=10):
        params_pass = self.params + self.consts
        model_partial = partial(self.model, params = params_pass)
        t_span = (0, t)
        t_eval = np.linspace(0, t, 250)
        sol = solve_ivp(model_partial, t_span = t_span, y0 = init_cond, t_eval = t_eval, rtol=1e-6, atol=1e-8)
        return sol
    
    def plot(self, init_cond, t=10):
        sol = self.simulate(init_cond, t)
        c = self.__calc_all(sol, self.consts)
        fig, ax = plt.subplots(3, 1, figsize = (8,10))
        i = 0
        for ax, y in zip(ax, c):
            ax.plot(sol.t, y, label = self.species[i])
            ax.legend()
            i += 1
        return fig, ax
    
    def interactive_plot(self, init_cond, param_num=4, t=10, range = 5, step=0.05):
        if param_num == 4:
            print('4 parameters')
            def plot_temp(alpha, beta, theta, phi):
                params_old = self.params
                params = [alpha, beta, theta, phi]
                self.params = params
                self.plot(init_cond, t)
                self.params = params_old
            
            alpha, beta, theta, phi = self.params
            alpha_slider = widgets.FloatSlider(value=alpha, min=max(0, alpha-range), max=alpha+range, step=step, description='alpha')
            beta_slider = widgets.FloatSlider(value=beta, min=max(0, beta-range), max=beta+range, step=step, description='beta')
            theta_slider = widgets.FloatSlider(value=theta, min=max(0, theta-range), max=theta+range, step=step, description='theta')
            phi_slider = widgets.FloatSlider(value=phi, min=max(0, phi-range), max=phi+range, step=step, description='phi')

            interactive_widget = widgets.interactive(plot_temp, alpha=alpha_slider, beta=beta_slider, theta=theta_slider, phi=phi_slider)
            display(interactive_widget)

        elif param_num == 5:
            print('5 parameters')
            pass

    def fit(self, model, exp_data):
        pass

In [224]:
params = [10, 10, 10, 10]
consts = [0.17, 4]
init_cond = [0.1, 0]

In [232]:
model_test = oscmodel(model1, params, consts)
model_test.info = '4 Parameters: alpha, beta, theta, phi; 2 Constants: lambda, m'
model_test.calc_all(calculate_all_conc)

In [ ]:
model_test.interactive_plot(init_cond, range=10, t=6)

4 parameters


interactive(children=(FloatSlider(value=10.0, description='alpha', max=20.0, step=0.05), FloatSlider(value=10.…